In [3]:
from itertools import product
import random
import numpy as np
from scipy import sparse
from copy import copy, deepcopy

In [4]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    random.seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random.random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[random.randint(0, num_sets-1), p] = True
    # for the return, thanks Beatrice Occhiena
    return np.array(sets.toarray())

num_points = [100, 1000, 5000]
num_sets = num_points
density = [0.3, 0.7]

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [5]:
x = make_set_covering_problem(5, 5, .3)
print("Element at row=0 and column=1:", x[0, 1])

Element at row=0 and column=1: False


The matrix generated by make_set_covering_problem() function is made of num_sets rows and num_points columns.
Each row is randomly generated and is made of True or False elements.
The objective is to pick the lowest amount of rows that, together, cover the biggest number of positions.
i.e.
* row 1 = True, False, False, True, False, True
* row 2 = False, True, True, False, False, False
* row 3 = True, False, False, False, False, False
* row 4 = False, False, False, False, True, True
* row 5 = False, False, True, True, False, True

If we pick rows 1, 2, 4, they cover with True all the positions

The **state** is an array of dimension num_sets that indicates which rows we took from the problem.
i.e. in the aforemensioned example:
* state = [True, True, False, True, False]

Because we solved the problem taking rows corresponding to indexes 0,1,3 of the problem matrix

We firstly need a fitness function and a tweak function

In [6]:
def fitness_old(state):
    return np.sum(state)

def fitness(state, sets):
    selected_sets = sets[state == 1]
    universe_coverage = np.sum(np.sum(selected_sets, axis=0) > 0)
    return universe_coverage

def fitness_beatrice(state, sets):
    """
    OB: Maximise the number of covered points, minimise the cost, minimise the overlap
    """
    cost = np.sum(state)
    selected_sets = sets[state == 1]
    overlap = np.sum(np.sum(selected_sets, axis=0) > 1)
    universe_coverage = np.sum(np.sum(selected_sets, axis=0) > 0)

    return universe_coverage, -cost, -overlap

def tweak(state):
    # we choose a random element of the array and flip the corresponding bit
    new_state = copy(state)
    index = random.randint(0, len(state) - 1)
    new_state[index] = not new_state[index]
    return new_state

## Hill-Climbing

* `Random Mutation HC`: It just iteratively test new candidate solutions in the region of the current candidate, and adopt the new ones if they're better.
* `Steepest ascent HC`: it generates multiple "tweaks" to a candidate solution simultaneously, and then it selects the best-performing one for adoption.
* `Random-restart HC`: In order to avoid getting stuck in a local optimum, we can restart the algorithm from a different random starting point. Note that we're just increasing the probability of finding the global optimum, but we're not guaranteed to find it.

In [24]:
def random_mutation_hc(initial_solution, problem, max_iterations):
    fitness_calls = 0
    number_iterations = 0

    actual_state = initial_solution
    actual_fitness = fitness(actual_state, problem)
    fitness_calls += 1

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1
        next_state = tweak(actual_state)
        next_fitness = fitness(next_state, problem)
        fitness_calls += 1

        # If the next state is better, update the problem
        if(next_fitness > actual_fitness):
            actual_state = next_state
            actual_fitness = next_fitness

            # If we have reached a satisfactory solution, stop
            if(actual_fitness == len(problem)):
                break

    return actual_state, number_iterations, fitness_calls

In [26]:
max_iterations = 1000

for n, d in product(num_points, density):
    print("n={}, d={}".format(n, d))
    problem = make_set_covering_problem(n, n, d)
    initial_solution = np.zeros(n, dtype=bool)
    rmhc, iterations, fitness_calls = random_mutation_hc(initial_solution, problem, max_iterations)
    print("RMHC: {} iterations".format(iterations))
    print("RMHC: {} fitness".format(fitness(rmhc, problem)))
    print("RMHC: {} fitness calls".format(fitness_calls))
    print("-------------------------")
    

n=100, d=0.3
RMHC: 11 iterations
RMHC: 100 fitness
RMHC: 12 fitness calls
-------------------------
n=100, d=0.7
RMHC: 4 iterations
RMHC: 100 fitness
RMHC: 5 fitness calls
-------------------------
n=1000, d=0.3
RMHC: 22 iterations
RMHC: 1000 fitness
RMHC: 23 fitness calls
-------------------------
n=1000, d=0.7
RMHC: 6 iterations
RMHC: 1000 fitness
RMHC: 7 fitness calls
-------------------------
n=5000, d=0.3
RMHC: 24 iterations
RMHC: 5000 fitness
RMHC: 25 fitness calls
-------------------------
n=5000, d=0.7
RMHC: 9 iterations
RMHC: 5000 fitness
RMHC: 10 fitness calls
-------------------------


In [30]:
def steepest_ascend_hc(initial_solution, problem, neighbours_size, max_iterations):
    number_iterations = 0
    fitness_calls = 0
    neighbours = []
    fitness_neighbours = []
    
    actual_state = initial_solution
    actual_fitness = fitness(actual_state, problem)
    fitness_calls += 1

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1

        for _ in range(neighbours_size):
            neighbours.append(tweak(actual_state))
            fitness_neighbours.append(fitness(neighbours[-1], problem))
            fitness_calls += 1

        best_neighbour = neighbours[np.argmax(fitness_neighbours)]
        best_neighbour_fitness = fitness_neighbours[np.argmax(fitness_neighbours)]


        # If the next state is better, update the problem
        if(best_neighbour_fitness > actual_fitness):
            actual_state = best_neighbour
            actual_fitness = best_neighbour_fitness

            # If we have reached a satisfactory solution, stop
            if(actual_fitness == len(problem)):
                break

    return actual_state, number_iterations, fitness_calls

In [31]:
for n, d in product(num_points, density):
    print("n={}, d={}".format(n, d))
    problem = make_set_covering_problem(n, n, d)
    initial_solution = np.zeros(n, dtype=bool)
    sahc, iterations, fitness_calls = steepest_ascend_hc(initial_solution, problem, 10, max_iterations)
    print("SAHC: {} iterations".format(iterations))
    print("SAHC: {} fitness".format(fitness(sahc, problem)))
    print("SAHC: {} fitness calls".format(fitness_calls))
    print("-------------------------")

n=100, d=0.3
SAHC: 8 iterations
SAHC: 100 fitness
SAHC: 81 fitness calls
-------------------------
n=100, d=0.7
SAHC: 3 iterations
SAHC: 100 fitness
SAHC: 31 fitness calls
-------------------------
n=1000, d=0.3
SAHC: 12 iterations
SAHC: 1000 fitness
SAHC: 121 fitness calls
-------------------------
n=1000, d=0.7
SAHC: 5 iterations
SAHC: 1000 fitness
SAHC: 51 fitness calls
-------------------------
n=5000, d=0.3
SAHC: 19 iterations
SAHC: 5000 fitness
SAHC: 191 fitness calls
-------------------------
n=5000, d=0.7
SAHC: 6 iterations
SAHC: 5000 fitness
SAHC: 61 fitness calls
-------------------------


In [33]:
def random_restart_hc(problem, restarts, max_iterations):
    number_of_restarts = 0
    fitness_calls = 0
    
    best_state = None
    
    # Loop until we reach the maximum number of restarts
    for _ in range(restarts):
        number_of_restarts += 1
        candidate_state = np.zeros(n, dtype=bool)
        candidate_state, fitness_candidate, fitness_calls_restarts = random_mutation_hc(candidate_state, problem, max_iterations)
        fitness_calls += fitness_calls_restarts

        if(best_state is None or fitness_candidate > fitness_best_state):
            best_state = candidate_state
            fitness_best_state = fitness_candidate

        # If we have reached a satisfactory solution, stop
        if(fitness_best_state == len(problem)):
            break

    return best_state, number_of_restarts, fitness_calls

In [34]:
max_restarts = 100

for n, d in product(num_points, density):
    print("n={}, d={}".format(n, d))
    problem = make_set_covering_problem(n, n, d)
    rrhc, iterations, fitness_calls = random_restart_hc(problem, max_restarts, max_iterations)
    print("RRHC: {} iterations".format(iterations))
    print("RRHC: {} fitness".format(fitness(rrhc, problem)))
    print("RRHC: {} fitness calls".format(fitness_calls))
    print("-------------------------")

n=100, d=0.3
RRHC: 100 iterations
RRHC: 100 fitness
RRHC: 1670 fitness calls
-------------------------
n=100, d=0.7
RRHC: 100 iterations
RRHC: 100 fitness
RRHC: 581 fitness calls
-------------------------
n=1000, d=0.3
RRHC: 100 iterations
RRHC: 1000 fitness
RRHC: 2305 fitness calls
-------------------------
n=1000, d=0.7
RRHC: 100 iterations
RRHC: 1000 fitness
RRHC: 783 fitness calls
-------------------------
n=5000, d=0.3
RRHC: 100 iterations
RRHC: 5000 fitness
RRHC: 2679 fitness calls
-------------------------
n=5000, d=0.7
RRHC: 100 iterations
RRHC: 5000 fitness
RRHC: 919 fitness calls
-------------------------


Comparison between RHMC and SAHC.

SAHC uses more fitness calls as expected, but less iterations than RHMC

In [35]:
max_iterations = 1000
neighbours_size = 10
max_restarts = 100

for n, d in product(num_points, density):
    print("n={}, d={}".format(n, d))
    problem = make_set_covering_problem(n, n, d)
    initial_solution = np.zeros(n, dtype=bool)
    rmhc, iterations_rmhc, fitness_calls_rhmc = random_mutation_hc(initial_solution, problem, max_iterations)
    sahc, iterations_sahc, fitness_calls_sahc = steepest_ascend_hc(initial_solution, problem, neighbours_size, max_iterations)
    # rrhc, iterations_rrhc, fitness_calls_rrhc = random_restart_hc(problem, max_restarts, max_iterations)
    print("RMHC: {} iterations".format(iterations_rmhc))
    print("RMHC: {} fitness".format(fitness(rmhc, problem)))
    print("RMHC: {} fitness calls".format(fitness_calls_rhmc))
    print("SAHC: {} iterations".format(iterations_sahc))
    print("SAHC: {} fitness".format(fitness(sahc, problem)))
    print("SAHC: {} fitness calls".format(fitness_calls_sahc))
    # print("RRHC: {} iterations".format(iterations_rrhc))
    # print("RRHC: {} fitness".format(fitness(rrhc, problem)))
    # print("RRHC: {} fitness calls".format(fitness_calls_rrhc))
    print("-------------------------")

n=100, d=0.3
RMHC: 11 iterations
RMHC: 100 fitness
RMHC: 12 fitness calls
SAHC: 7 iterations
SAHC: 100 fitness
SAHC: 71 fitness calls
-------------------------
n=100, d=0.7
RMHC: 4 iterations
RMHC: 100 fitness
RMHC: 5 fitness calls
SAHC: 3 iterations
SAHC: 100 fitness
SAHC: 31 fitness calls
-------------------------
n=1000, d=0.3
RMHC: 22 iterations
RMHC: 1000 fitness
RMHC: 23 fitness calls
SAHC: 12 iterations
SAHC: 1000 fitness
SAHC: 121 fitness calls
-------------------------
n=1000, d=0.7
RMHC: 6 iterations
RMHC: 1000 fitness
RMHC: 7 fitness calls
SAHC: 5 iterations
SAHC: 1000 fitness
SAHC: 51 fitness calls
-------------------------
n=5000, d=0.3
RMHC: 24 iterations
RMHC: 5000 fitness
RMHC: 25 fitness calls
SAHC: 16 iterations
SAHC: 5000 fitness
SAHC: 161 fitness calls
-------------------------
n=5000, d=0.7
RMHC: 9 iterations
RMHC: 5000 fitness
RMHC: 10 fitness calls
SAHC: 6 iterations
SAHC: 5000 fitness
SAHC: 61 fitness calls
-------------------------


### Simulated-Annealing

This technique is based on performing exploration, and then exploitation, based on an hyperparameter, the temperature, that allows to accept a worse candidate solution with a certain probability.
This is useful to escape from local minima

The probability of accepting a new candidate solution is defined by the `Boltzmann distribution`:
- $S$ = current solution
- $R$ = new candidate solution
- $t$ = current temperature
- $c$ = cooling rate $\in [0,1]$ that defines how fast the temperature decreases

$P(R|S,t) = e^{\frac{Quality(R)-Quality(S)}{t}}$

In [ ]:
def simulated_annealing(problem, temperature, cooling_rate, max_iterations):
    